In [1]:
import pandas as pd
import geojson

In [2]:
from geojson import Feature, Point, FeatureCollection

houses_df = pd.read_csv('../data_4/property_4_lat_lon.csv')  
police_df = pd.read_csv('../data_4/api/victoria_police_stations.csv') 
houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
police_df = police_df.dropna(subset=['longitude', 'latitude'])


### create GeoJSON Feature 
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

### convert the data into Geojson
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

police_features = police_df.apply(lambda row: create_geojson_feature(row, 'police'), axis=1).tolist()
police_feature_collection = FeatureCollection(police_features)


### save the geo file
houses_geojson_output_path = '../data_4/api/property_4_geojson.geojson'
police_geojson_output_path = '../data_4/api/victoria_police_stations_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(police_geojson_output_path, 'w') as f:
    geojson.dump(police_feature_collection, f)


print("Geojson created")


房屋和车站的GeoJSON文件已生成


In [3]:
import geojson
import openrouteservice
import pandas as pd
import time

### api client
api_key = '5b3ce3597851110001cf6248c2b2602de2dc43afa9f186cdfc39caca' 
client = openrouteservice.Client(key=api_key)

### read Geojson
with open('../data_4/api/property_4_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data_4/api/victoria_police_stations_geojson.geojson', 'r') as f:
    police_geojson = geojson.load(f)

### create the function to get the latitude and longtitude
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
police_coordinates = extract_coordinates(police_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
police_coords = [(lon, lat) for lat, lon, _ in police_coordinates]


### using api to calculate the driving distance
def calculate_distance_matrix_in_batches(house_coords, police_coords, batch_size=18):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + police_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(police_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            ### find mini distance
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_police_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_police = police_coordinates[closest_police_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_police': closest_police,
                        'minimum_distance_police': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1) 
    
    return all_shortest_distances

### batch size the data
shortest_distances = calculate_distance_matrix_in_batches(house_coords, police_coords, batch_size=18)

### save data
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data_4/api/results/closest_police_distance_final_4.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved into {output_path}")


结果已保存到 ../data_4/api/results/closest_police_distance_final_4.csv
